In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
import seaborn as sns
import scipy
import scipy.stats

In [75]:
#####   F U N C T I O N S

### generic functions

def PolyFit(x, deg, wlo, scn, col, n=3, figs=False):
    wlo = wlo.dropna(subset=[col])
    
    ### fourth order polyfit
    poly = np.poly1d(np.polyfit(wlo.index, wlo[col], deg))
    
    ### plot
    plt.plot(x, poly(x), 'r-',
             list(wlo.index)[:-n], wlo[col][:-n], 'b.', 
             list(wlo.index)[-n:], wlo[col][-n:], 'g*', 
            )
#     for mode in list(wlo.columns[1:]):
#         plt.plot(list(wlo.index), wlo[mode],'.')
    plt.legend(('interpolation','historical','WLO'), loc='right')
    if scn=='HP' or scn=='LP':
        plt.ylabel('Person-kilometers')
    if scn=='HF' or scn=='LF':
        plt.ylabel('Ton-kilometers')
    plt.ylim(0,1.1*max(poly(x)))
    plt.savefig(str('figures/WLO_Service_'\
                    +scn+'_'
                    +str(int(min(x)))\
                    +'-'+str(int(max(x)))\
                    +'.png'), dpi=400)
    if figs is True:
        plt.show()
    plt.clf()
    return poly(x)


def InnoDiff(x, start=0, end=1, steepness=1, midpoint=2030):
    if end < start:
        y = np.ones(len(x))*start - np.ones(len(x))*(start-end) / (1+np.exp(-steepness*(x-midpoint)))
    elif end >= start: 
        y = np.ones(len(x))*start + np.ones(len(x))*(end-start) / (1+np.exp(-steepness*(x-midpoint)))
    return y


def BinaryShifts(x, D, mode, figs=False):
    for key in D.keys():
        pal = sns.color_palette("Set1")
        y = [D[key], 1-D[key] ]
        plt.stackplot(x, y, colors=pal, alpha=0.6 )
        plt.savefig(str('figures/WLOHP_Share'\
                        +'_'+mode
                        +key
                        +'.png'), dpi=300)
        if figs is True:
            plt.show()
        plt.clf()

    
def PlotService(x, df, y='Share', 
                scn='HoogPerson', 
                figs=False, 
                savefig='pdf', 
                color='Mode',
                group='Vehicle',
                ):
    fig = px.area(df, x = 'Year', y = y, 
                      color = color, 
                      line_group = group,
                      width = 800,
                      height = 500,
                      ).update_layout(legend=dict(
                                                  y=0.5, 
                                                  traceorder='reversed', 
                                                  font_size=10,
                                                  ))
    if figs is True:
        fig.show()
        
    fig.write_image(str('figures/WLO_'\
                                +scn+'_'\
                                +y+'_'\
                                +str(int(min(x)))\
                                +'-'+str(int(max(x)))\
                                +'.'+savefig))


def StackPlot(x, a, b, path, figs=False):
    plt.stackplot(x, a, b)
    plt.savefig(str('figures/'+path), dpi=300)
    if figs is True:
        plt.show()
    plt.clf()
    
    
def WeibCorr(x, start=2000, scale=10, shape=2, magn=1e3):
    weibcorr = magn * scipy.stats.weibull_min.pdf(x,shape, loc=start, scale=scale)
    return weibcorr


###   PERSON TRANSPORT

def CarsFirst(x, poly):
    df = pd.DataFrame()
    keys = ['ice', 'ev', 'hev', 'train', 'metro-bus', 'ebike', 'bike', 'emoped', 'moped', 'walk']
    for key in keys:
        temp = pd.DataFrame()
        temp['Year'] = x
        temp['Vehicle'] = key
        temp['Share'] = np.nan
        df = pd.concat([df, temp], ignore_index=True, sort=False)
    
    D = dict()
        
    ### for drive first

    D['drive'] = InnoDiff(x, start=0.76, end=0.1,
                          steepness=0.25, midpoint=2035)

    D['ice'] = InnoDiff(x,start=0.99,end=0.15,
                        steepness=0.3,midpoint=2030)

    D['ev'] = InnoDiff(x, start=0.01, end=0.95,
                   steepness=0.3, midpoint=2025)

    D['public'] = InnoDiff(x, start=0.50, end=0.72,
                     steepness=0.2, midpoint=2030 )

    D['train'] = InnoDiff(x, start=0.77, end=0.77,
                      steepness=1,  midpoint=2040 )

    D['cycle'] = InnoDiff(x, start=0.80, end=0.80,
                     steepness=1, midpoint=2035 )

    D['bicycle'] = InnoDiff(x, start=0.93, end=0.85,
                     steepness=0.3, midpoint=2030 )

    D['bike'] = InnoDiff(x, start=1, end=0.8,
                     steepness=0.25, midpoint=2022 )

    D['moped'] = InnoDiff(x, start=0.999, end=0.001,
                      steepness=0.3, midpoint=2025 )
    
    ones = np.ones(len(x))
    ### cars
    df.loc[df['Vehicle']=='ice', 'Share'] = D['drive'] * D['ice']
    df.loc[df['Vehicle']=='ev', 'Share'] = D['drive'] * (ones - D['ice']) * D['ev']
    df.loc[df['Vehicle']=='hev', 'Share'] = D['drive'] * (ones - D['ice'])* (ones - D['ev'])

    ### public transport
    df.loc[df['Vehicle']=='train', 'Share'] = (ones - D['drive']) * D['public'] * D['train']
    df.loc[df['Vehicle']=='metro-bus', 'Share'] = (ones - D['drive']) * D['public'] * (ones - D['train'])

    ### slow transport
    df.loc[df['Vehicle']=='bike', 'Share'] = (ones - D['drive']) * (ones - D['public']) * D['cycle'] * D['bicycle'] * D['bike']
    df.loc[df['Vehicle']=='ebike', 'Share'] = (ones - D['drive']) * (ones - D['public']) * D['cycle'] * D['bicycle'] * (ones - D['bike'])
    df.loc[df['Vehicle']=='moped', 'Share'] = (ones - D['drive']) * (ones - D['public']) * D['cycle'] * (ones - D['bicycle']) * D['moped']
    df.loc[df['Vehicle']=='emoped', 'Share'] = (ones - D['drive']) * (ones - D['public']) * D['cycle'] * (ones - D['bicycle']) * (ones - D['moped'])
    df.loc[df['Vehicle']=='walk', 'Share'] = (ones - D['drive']) * (ones - D['public']) * (ones - D['cycle'])
    
    df['Person-kilometers'] = None
    for i in df['Vehicle'].unique():
        df.loc[df['Vehicle']==i, 'Person-kilometers'] = df.loc[df['Vehicle']==i, 'Share'].multiply(poly)
        
    modemap = {'Driving' : ['ice', 'ev', 'hev'],
               'Transit' : ['train', 'metro-bus'],
               'Cycling' : ['ebike', 'bike', 'moped', 'emoped'],
               'Walking' : ['walk'],
               }
    for key in list(modemap.keys()):
        for i in range(len(modemap[key])):
            df.loc[df['Vehicle']==modemap[key][i], 'Mode'] = key
    
    for each in df['Mode'].unique():
        print(each, round(df.loc[df['Year']==2050]\
                            .loc[df['Mode']==each,'Share'].sum(),3)\
              )
    return df, D


def ServiceToPersonVehicles(x, df, service='Person-kilometers', figs=False):
    STV = dict()
    keys = ['ice', 'ev', 'hev', 'train', 'metro-bus', 'ebike', 'bike', 'emoped', 'moped', 'walk']
    
    ### cars per person kilometer
    CarPerPkm = 6.34e6/136.5
    CarCurve = InnoDiff(x, start=CarPerPkm, end=CarPerPkm*0.5,
                        steepness=0.3, midpoint=2030 )
    CarCorr = WeibCorr(x, start=2000, scale=20, shape=3, magn=2.3e5)
    STV['ice'] = CarCurve + CarCorr 
    STV['ev'] = STV['ice']
    STV['hev'] = STV['ice']
    StackPlot(x, CarCurve, CarCorr, 'CarsPerPersonkm.png', figs=False)
    
    ### bikes per person kilometer, assumed to be constant
    BikePerPkm = 17.8e6/15.5/20*19
    BikeCurve = InnoDiff(x, start=BikePerPkm, end=BikePerPkm*0.75,
                      steepness=0.3, midpoint=2030 )
    BikeCorr = WeibCorr(x, start=2000, scale=20, shape=3, magn=5e6)
    STV['bike'] = BikeCurve +  BikeCorr
    STV['ebike'] = STV['bike']
    StackPlot(x, BikeCurve, BikeCorr, 'BikesPerPersonkm.png', figs=False)
    
    ### train capacity per person kilometer, assumed to be constant
    TrainPerPkm = 1301/17.1
    TrainCurve = InnoDiff(x, start=TrainPerPkm, end=TrainPerPkm,
                      steepness=1, midpoint=2035 )
    STV['train'] = TrainCurve

    ### buses per person kilometer, assumed to be constant
    BusPerPkm = 11634/6.7
    BusCurve = InnoDiff(x, start=BusPerPkm, end=BusPerPkm,
                      steepness=1, midpoint=2035 )
    STV['metro-bus'] = BusCurve
    
    ### mopeds per person kilometer, assumed to be constant
    MopedPerPkm = 0.7e6/1
    MopedCurve = InnoDiff(x, start=MopedPerPkm, end=MopedPerPkm*0.3,
                      steepness=0.3, midpoint=2035 )
    STV['moped'] = MopedCurve
    STV['emoped'] = MopedCurve
    
    ###   M A T H 
    df['VehicleCount'] = None
    for key in STV.keys():
        df.loc[df['Vehicle']==key, 'VehicleCount'] = STV[key] * df.loc[df['Vehicle']==key, service]
        
    return df, STV


###   FREIGHT TRANSPORT

def RoadFirst(x, poly):
    df = pd.DataFrame()
    keys = ['40tlorry', '28tlorry', '16tlorry', 'freighttrain', 'xlbarge', 'lbarge', 'mbarge', 'sbarge']
    for key in keys:
        temp = pd.DataFrame()
        temp['Year'] = x
        temp['Vehicle'] = key
        temp['Share'] = np.nan
        df = pd.concat([df, temp], ignore_index=True, sort=False)
        
    D = dict()
    D['ROAD'] = InnoDiff(x, start=0.60, end=0.29,
                         steepness=0.2, midpoint=2035)
    D['16TL'] = InnoDiff(x, start=0.06, end=0.10,
                         steepness=0.2, midpoint=2030)
    D['40TL'] = InnoDiff(x, start=0.22, end=0.40,
                         steepness=0.2, midpoint=2030)
    D['RAIL'] = InnoDiff(x, start=0.11, end=0.4,
                         steepness=0.2, midpoint=2030)
    D['SBARGE'] = InnoDiff(x, start=0.47, end=0.2,
                         steepness=0.1, midpoint=2030)
    D['MBARGE'] = InnoDiff(x, start=0.45, end=0.2,
                         steepness=0.1, midpoint=2030)
    D['LBARGE'] = InnoDiff(x, start=0.60, end=0.4,
                         steepness=0.1, midpoint=2030)
    
    
    ones = np.ones(len(x))
    df.loc[df['Vehicle']=='16tlorry', 'Share'] = D['ROAD'] * D['16TL']
    df.loc[df['Vehicle']=='28tlorry', 'Share'] = D['ROAD'] * (ones - D['16TL']) * (ones - D['40TL'])
    df.loc[df['Vehicle']=='40tlorry', 'Share'] = D['ROAD'] * (ones - D['16TL']) * D['40TL']

    df.loc[df['Vehicle']=='freighttrain', 'Share'] = (ones - D['ROAD']) * D['RAIL']
    
    df.loc[df['Vehicle']=='sbarge', 'Share'] = (ones - D['ROAD']) * (ones - D['RAIL']) * D['SBARGE']
    df.loc[df['Vehicle']=='mbarge', 'Share'] = (ones - D['ROAD']) * (ones - D['RAIL']) * (ones - D['SBARGE']) * D['MBARGE'] 
    df.loc[df['Vehicle']=='lbarge', 'Share'] = (ones - D['ROAD']) * (ones - D['RAIL']) * (ones - D['SBARGE']) * (ones - D['MBARGE']) * D['LBARGE'] 
    df.loc[df['Vehicle']=='xlbarge', 'Share'] = (ones - D['ROAD']) * (ones - D['RAIL']) * (ones - D['SBARGE']) * (ones - D['MBARGE']) * (ones - D['LBARGE'])
    
    df['Ton-kilometers'] = None
    for i in df['Vehicle'].unique():
        df.loc[df['Vehicle']==i, 'Ton-kilometers'] = df.loc[df['Vehicle']==i, 'Share'].multiply(poly)
    
    modemap = {'Road' : ['40tlorry', '28tlorry', '16tlorry'],
               'Rail' : ['freighttrain'],
               'Inland' : ['xlbarge', 'lbarge', 'mbarge', 'sbarge'],
               }
    for key in list(modemap.keys()):
        for i in range(len(modemap[key])):
            df.loc[df['Vehicle']==modemap[key][i], 'Mode'] = key
    
    for each in df['Mode'].unique():
        print(each, round(df.loc[df['Year']==2050]\
                            .loc[df['Mode']==each,'Share'].sum(),3)\
              )
        
    return df, D


def ServiceToFreightVehicles(x, df, service='Ton-kilometers', figs=False):
    STV = dict()
    keys = ['40tlorry', '28tlorry', '16tlorry', 'freighttrain', 'xlbarge', 'lbarge', 'mbarge', 'sbarge']
    
    ### lorries per ton kilometer
    lorry16PerT = 240000/54.1*28/16
    lorry28PerT = 240000/54.1*28/28
    lorry40PerT = 240000/54.1*28/40
    L16Curve = InnoDiff(x, start=lorry16PerT, end=lorry16PerT,
                        steepness=1, midpoint=2035 )
    L28Curve = InnoDiff(x, start=lorry28PerT, end=lorry28PerT,
                        steepness=1, midpoint=2035 )
    L40Curve = InnoDiff(x, start=lorry40PerT, end=lorry40PerT,
                        steepness=1, midpoint=2035 )
    STV['16tlorry'] = L16Curve 
    STV['28tlorry'] = L28Curve
    STV['40tlorry'] = L40Curve
    
    ### trains per ton-kilometer
    trainPerT = 1000/5.9
    trainCurve = InnoDiff(x, start=trainPerT, end=trainPerT,
                        steepness=1, midpoint=2035 )
    STV['freighttrain']  = trainCurve

    ### inland barges per ton-kilometer
    XlbPerT = 623/46.6
    LbPerT = 943/46.6
    MbPerT = 1280/46.6
    SbPerT = 2536/46.6
    xlbCurve = InnoDiff(x, start=XlbPerT, end=XlbPerT, steepness=1, midpoint=2035 )
    lbCurve = InnoDiff(x, start=LbPerT, end=LbPerT, steepness=1, midpoint=2035 )
    mbCurve = InnoDiff(x, start=MbPerT, end=MbPerT, steepness=1, midpoint=2035 )
    sbCurve = InnoDiff(x, start=SbPerT, end=SbPerT, steepness=1, midpoint=2035 )
    STV['xlbarge']  = xlbCurve
    STV['lbarge']  = lbCurve  
    STV['mbarge']  = mbCurve  
    STV['sbarge']  = sbCurve  
    
    ### for output
    df['VehicleCount'] = None
    for key in STV.keys():
        df.loc[df['Vehicle']==key, 'VehicleCount'] = STV[key] * df.loc[df['Vehicle']==key, service]
        
    return df, STV


###   FLIGHT TRANSPORT

def Flights(x, poly):
    df = pd.DataFrame()
    keys = ['A330', 'B787']
    for key in keys:
        temp = pd.DataFrame()
        temp['Year'] = x
        temp['Vehicle'] = key
        temp['Share'] = np.nan
        df = pd.concat([df, temp], ignore_index=True, sort=False)
        
    D = dict()
    D['A330'] = InnoDiff(x, start=1, end=0.10,
                         steepness=0.2, midpoint=2030)    
    
    ones = np.ones(len(x))
    df.loc[df['Vehicle']=='A330', 'Share'] = D['A330'] 
    df.loc[df['Vehicle']=='B787', 'Share'] = (ones - D['A330'])
    
    key = 'Person-movements'
    df[key] = None
    for i in df['Vehicle'].unique():
        df.loc[df['Vehicle']==i, key] = df.loc[df['Vehicle']==i, 'Share'].multiply(poly)
    
    modemap = {'Air' : ['A330', 'B787'],
               }
    for key in list(modemap.keys()):
        for i in range(len(modemap[key])):
            df.loc[df['Vehicle']==modemap[key][i], 'Mode'] = key
    
    ### prints mode shares at last timestep
    for each in df['Mode'].unique():
        print(each, round(df.loc[df['Year']==2050]\
                            .loc[df['Mode']==each,'Share'].sum(),3)\
              )
        
    return df, D

def ServiceToFlightVehicles(x, df, service='Person-movements', figs=False):
    STV = dict()
    keys = ['A330', 'B787']
    
    ### aircraft per passenger movements 
    PM = np.array(df.groupby('Year')['Person-movements'].sum())

    ### for output
    df['VehicleCount'] = None
    df.at[df['Vehicle']=='A330', 'VehicleCount'] = (145 + 1.13 * PM) * df.loc[df['Vehicle']=='A330', 'Share']#STV[key] * df.loc[df['Vehicle']==key, service]
    df.at[df['Vehicle']=='B787', 'VehicleCount'] = (145 + 1.13 * PM) * df.loc[df['Vehicle']=='B787', 'Share']#STV[key] * df.loc[df['Vehicle']==key, service]

    return df, STV

###   SEA VESSEL TRANSPORT

def SeaVessels(x, poly, driver='SeaTGW'):

    shares = pd.read_csv('data/modalshift/ships.csv', header=0, index_col=None)
    keys = list(shares['Type'].unique())

    df = pd.DataFrame()
    for key in keys:
        temp = pd.DataFrame()
        temp['Year'] = x
        temp['Vehicle'] = key
        temp['Share'] = np.nan
        df = pd.concat([df, temp], ignore_index=True, sort=False)
    
    D = dict.fromkeys(shares['Type'].unique())
    for key in D.keys():
        D[key] = shares.loc[shares['Type']==key, 'Share'].values * np.ones(len(x))
        df.at[df['Vehicle']==key, 'Share'] = list(D[key])

    driver = 'SeaTGW'
    df[driver] = None
    for i in df['Vehicle'].unique():
        df.loc[df['Vehicle']==i, driver] = df.loc[df['Vehicle']==i, 'Share'].multiply(poly)
    
    modemap = {'Sea vessels' : keys,
               }
    for key in list(modemap.keys()):
        for i in range(len(modemap[key])):
            df.loc[df['Vehicle']==modemap[key][i], 'Mode'] = key
    
    ### prints mode shares at last timestep
    for each in df['Mode'].unique():
        print(each, round(df.loc[df['Year']==2050]\
                            .loc[df['Mode']==each,'Share'].sum(),3)\
              )
        
    return df, D

def ServiceToSeaVehicles(x, df, service='SeaTGW', figs=False):
    STV = dict()
    shares = pd.read_csv('data/modalshift/ships.csv', header=0, index_col=None)
    keys = list(shares['Type'].unique())

    ### sea vessel per SeaTGW
    GDP = np.array(df.groupby('Year')['SeaTGW'].sum())

    ### for output
    df['VehicleCount'] = None
    for key in keys:
        df.at[df['Vehicle']==key, 'VehicleCount'] = (1400 / 6070 * GDP) * df.loc[df['Vehicle']==key, 'Share']

    return df, STV

In [94]:
### set x-axis and fidelity.
startmodel = 1999
endmodel = 2051
x = np.linspace(startmodel, endmodel, endmodel-startmodel+1)

### read data
wloHP = pd.read_csv('data/modalshift/WLOHpassenger.csv', index_col=0,header=0)
wloLP = pd.read_csv('data/modalshift/WLOLpassenger.csv', index_col=0,header=0)
wloHF = pd.read_csv('data/modalshift/WLOHfreight.csv', index_col=0,header=0)
wloLF = pd.read_csv('data/modalshift/WLOLfreight.csv', index_col=0,header=0)
wloLSV = pd.read_csv('data/modalshift/SeaL.csv', index_col=0,header=0)
wloHSV = pd.read_csv('data/modalshift/SeaH.csv', index_col=0,header=0)

### convert to numeric column
### force polynomials to flatten out at 2050.
wloHP.loc[2040,'Total']=250
wloLP.loc[2040,'Total']=210
wloHF.loc[2040,'Total']=170
wloLF.loc[2040,'Total']=140
wloHP.loc[2040,'Luchtvaart']=140

### "PolyXX" is the service demand over time.
PolyHP = PolyFit(x, 4, wloHP, 'HP', 'Total')
PolyLP = PolyFit(x, 4, wloLP, 'LP', 'Total')
PolyHF = PolyFit(x, 4, wloHF, 'HF', 'Total')
PolyLF = PolyFit(x, 4, wloLF, 'LF', 'Total')
PolyLAC = PolyFit(x, 1, wloLP, 'LAC', 'Luchtvaart')
PolyHAC = PolyFit(x, 4, wloHP, 'HAC', 'Luchtvaart')
PolyLSV = PolyFit(x, 2, wloLSV, 'LSV', 'SeaGDP', n=32)
PolyHSV = PolyFit(x, 2, wloHSV, 'HSV', 'SeaGDP', n=32)

<Figure size 432x288 with 0 Axes>

In [95]:
###    R U N   F U N C T I O N S   P E R S O N   T R A N S P O R T 

### H I G H   S C E N A R I O 
dfHP, DHP = CarsFirst(x, PolyHP)
BinaryShifts(x, DHP, 'HoogPerson', figs=False)
dfHP, STVhp = ServiceToPersonVehicles(x, dfHP, figs=True)
PlotService(x, dfHP, y='Share', scn='HoogPerson', figs=False)
PlotService(x, dfHP, y='Person-kilometers', scn='HoogPerson', figs=False)
PlotService(x, dfHP, y='VehicleCount', scn='HoogPerson', figs=False)

### L O W   S C E N A R I O 
dfLP, DLP = CarsFirst(x, PolyLP)
BinaryShifts(x, DLP, 'LaagPerson', figs=False)
dfLP, STVlp = ServiceToPersonVehicles(x, dfLP, figs=False)
PlotService(x, dfLP, y='Share', scn='LaagPerson', figs=False)
PlotService(x, dfLP, y='Person-kilometers', scn='LaagPerson', figs=True)
PlotService(x, dfLP, y='VehicleCount', scn='LaagPerson', figs=True)


Driving 0.115
Transit 0.634
Cycling 0.201
Walking 0.05
Driving 0.115
Transit 0.634
Cycling 0.201
Walking 0.05


<Figure size 432x288 with 0 Axes>

In [96]:
###    R U N   F U N C T I O N S   F R E I G H T   T R A N S P O R T 

### H I G H   S C E N A R I O 
dfHF, DHF = RoadFirst(x, PolyHF)
BinaryShifts(x, DHF, 'HoogFreight', figs=False)
dfHF, STVhf = ServiceToFreightVehicles(x, dfHF, figs=False)
PlotService(x, dfHF, y='Share', scn='HoogFreight', figs=False)
PlotService(x, dfHF, y='Ton-kilometers', scn='HoogFreight', figs=False)
PlotService(x, dfHF, y='VehicleCount', scn='HoogFreight', figs=False)

### L O W   S C E N A R I O 
dfLF, DLF = RoadFirst(x, PolyLF)
BinaryShifts(x, DLF, 'LaagFreight', figs=False)
dfLF, STVlf = ServiceToFreightVehicles(x, dfLF, figs=False)
PlotService(x, dfLF, y='Share', scn='LaagFreight', figs=False)
PlotService(x, dfLF, y='Ton-kilometers', scn='LaagFreight', figs=True)
PlotService(x, dfLF, y='VehicleCount', scn='LaagFreight', figs=True)

Road 0.305
Rail 0.274
Inland 0.421
Road 0.305
Rail 0.274
Inland 0.421


<Figure size 432x288 with 0 Axes>

In [97]:
### F L I G H T   M O D E L I N G 

### high scenario
dfHAC, DHAC = Flights(x, PolyHAC)
BinaryShifts(x, DHAC, 'HoogAir', figs=False)
dfHAC, STVhac = ServiceToFlightVehicles(x, dfHAC, service='Person-movements', figs=False)
PlotService(x, dfHAC, y='Share', scn='HoogAir', figs=False)
PlotService(x, dfHAC, y='Person-movements', scn='HoogAir', figs=False)
PlotService(x, dfHAC, y='VehicleCount', scn='HoogAir', figs=False, color='Vehicle')

### low scenario
dfLAC, DLAC = Flights(x, PolyLAC)
BinaryShifts(x, DLAC, 'LaagAir', figs=False)
dfLAC, STVlac = ServiceToFlightVehicles(x, dfLAC, service='Person-movements', figs=False)
PlotService(x, dfLAC, y='Share', scn='LaagAir', figs=False)
PlotService(x, dfLAC, y='Person-movements', scn='LaagAir', figs=False)
PlotService(x, dfLAC, y='VehicleCount', scn='LaagAir', figs=True, color='Vehicle')


Air 1.0
Air 1.0


<Figure size 432x288 with 0 Axes>

In [98]:
### S E A   M O D E L I N G 

### high scenario
dfHSV, DHSV = SeaVessels(x, PolyHSV)
BinaryShifts(x, DHSV, 'HoogTGW', figs=False)
dfHSV, STVhsv = ServiceToSeaVehicles(x, dfHSV, service='SeaTGW', figs=False)
PlotService(x, dfHSV, y='Share', scn='HoogSea', figs=False)
PlotService(x, dfHSV, y='SeaTGW', scn='HoogSea', figs=False)
PlotService(x, dfHSV, y='VehicleCount', scn='HoogSea', figs=False, color='Vehicle')

### low scenario
dfLSV, DLSV = SeaVessels(x, PolyLSV)
BinaryShifts(x, DLSV, 'LaagTGW', figs=False)
dfLSV, STVlsv = ServiceToSeaVehicles(x, dfLSV, service='SeaTGW', figs=False)
PlotService(x, dfLSV, y='Share', scn='LaagSea', figs=False)
PlotService(x, dfLSV, y='SeaTGW', scn='LaagSea', figs=False)
PlotService(x, dfLSV, y='VehicleCount', scn='LaagSea', figs=True, color='Vehicle')


Sea vessels 1.0
Sea vessels 1.0


<Figure size 432x288 with 0 Axes>

In [125]:
### combines all dataframes to a single Vehicle Count dataframe
vc = pd.DataFrame() 
for df in [dfLP, dfHP, dfLF, dfHF, dfLAC, dfHAC, dfLSV, dfHSV]:
    vc = pd.concat([vc, df[['Year', 'Vehicle', 'VehicleCount']]], ignore_index=True, sort=False)

### dictionary, VehicleInOutStock, to gather all stock and flow data
vios = dict()
for veh in list(vc['Vehicle'].unique()):
    vios[veh] = dict.fromkeys(['i', 'o', 's'])
    vios[veh]['s'] = np.array(vc.loc[vc['Vehicle']==veh, 'VehicleCount'])
    
lifetime = pd.read_csv('data/cdf/lifetime.csv', header=0, index_col=None)
for v in vios.keys():
    vios[v]['lifetime'] = np.ones(len(x))*lifetime.loc[lifetime['Vehicle']==v, 'lifetime'].values
    vios[v]['shape'] = lifetime.loc[lifetime['Vehicle']==v, 'shape'].values
    
del vios['walk']

In [126]:
vios
# list(vc['Vehicle'].unique())


{'ice': {'i': None,
  'o': None,
  's': array([6440562.719563214, 6403781.908922897, 6386661.6997586405,
         6400292.323507054, 6443777.148322556, 6515996.687968366,
         6615414.701344331, 6739893.58264069, 6886528.12187775,
         7051509.388175055, 7230032.463669394, 7416261.702315232,
         7603364.80829733, 7783622.750012161, 7948615.001548137,
         8089470.78839089, 8197167.048569856, 8262845.346804469,
         8278115.115795416, 8235312.840741798, 8127699.993800602,
         7949609.287553707, 7696590.5322767785, 7365658.453883715,
         6955788.528427015, 6468805.823111068, 5910709.086814305,
         5293214.173091822, 4634909.258046058, 3961083.927768294,
         3301399.804835729, 2685403.258908506, 2137173.7304519513,
         1671222.2253937188, 1291269.2663204584, 991995.3787686849,
         762472.7000392147, 589661.3584432289, 460947.5248610849,
         365493.33752653864, 294658.4359936152, 241879.13914588225,
         202316.38899951742, 172460

In [127]:
# np.ones(len(x))*lifetime.loc[3, 'lifetime']
# v='ice'
# lifetime.loc[lifetime['Vehicle']==v, 'shape'].values

In [128]:
import stockflow as sf

In [129]:
for v in vios.keys():
    IOS, dt = sf.InOutStock(\
                            x,
                            vios[v]['s'],
                            vios[v]['lifetime'],
                            shape = vios[v]['shape'],
                            scaleflow = 'year',
                            )
    vios[v]['i'] = np.array(IOS['Infl'])
    vios[v]['o'] = np.array(IOS['Outf'])
    
                

In [163]:
temp = pd.DataFrame()
key='ice'
temp.at[:, 'Year'] = pd.Series(x)
temp.at[:, 'Vehicle'] = key
temp.at[:, 'Stock'] = pd.Series(vios[key]['s'])
temp.at[:, 'Inflow'] = pd.Series(vios[key]['i'])
temp.at[:, 'Outflow'] = pd.Series(vios[key]['o'])

temp.head(15)

,Year,Vehicle,Stock,Inflow,Outflow
0,1999.0,ice,6.44056e+06,303869.749155,340650.559796
1,2000.0,ice,6.40378e+06,315004.323485,332124.532649
2,2001.0,ice,6.38666e+06,341915.021416,328284.397668
3,2002.0,ice,6.40029e+06,370128.784701,326643.959886
4,2003.0,ice,6.44378e+06,398416.281056,326196.741410
5,2004.0,ice,6.516e+06,425790.462400,326372.449024
6,2005.0,ice,6.61541e+06,451325.917748,326847.036452
7,2006.0,ice,6.73989e+06,474061.797868,327427.258631
8,2007.0,ice,6.88653e+06,492984.488872,328003.222574
9,2008.0,ice,7.05151e+06,507042.865376,328519.789882


In [164]:
VIOS = pd.DataFrame()
for key in vios.keys():
    temp = pd.DataFrame()
    temp.at[:, 'Year'] = pd.Series(x)
    temp.at[:, 'Vehicle'] = key
    temp.at[:, 'Stock'] = pd.Series(vios[key]['s'])
    temp.at[:, 'Inflow'] = pd.Series(vios[key]['i'])
    temp.at[:, 'Outflow'] = pd.Series(vios[key]['o'])
    VIOS = pd.concat([VIOS, temp], ignore_index=True, sort=False)
    
VIOS

,Year,Vehicle,Stock,Inflow,Outflow
0,1999.0,ice,6.44056e+06,303869.749155,340650.559796
1,2000.0,ice,6.40378e+06,315004.323485,332124.532649
2,2001.0,ice,6.38666e+06,341915.021416,328284.397668
3,2002.0,ice,6.40029e+06,370128.784701,326643.959886
4,2003.0,ice,6.44378e+06,398416.281056,326196.741410
5,2004.0,ice,6.516e+06,425790.462400,326372.449024
6,2005.0,ice,6.61541e+06,451325.917748,326847.036452
7,2006.0,ice,6.73989e+06,474061.797868,327427.258631
8,2007.0,ice,6.88653e+06,492984.488872,328003.222574
9,2008.0,ice,7.05151e+06,507042.865376,328519.789882


In [63]:
# dfHF.loc[dfHF['Year'].isin([2000, 2010, 2018, 2030, 2050])]\
#     .loc[:, ['Year', 'Vehicle','VehicleCount']]\
#     .pivot(index='Vehicle', columns='Year', values='VehicleCount')

In [64]:
# dfLF.loc[dfLF['Year'].isin([2000, 2010, 2018, 2030, 2050])]\
#     .loc[:, ['Year', 'Vehicle','VehicleCount']]\
#     .pivot(index='Vehicle', columns='Year', values='VehicleCount')

In [65]:
# dfHP.loc[dfHP['Year'].isin([2000, 2010, 2018, 2030, 2050])]\
#     .loc[:, ['Year', 'Vehicle','VehicleCount']]\
#     .pivot(index='Vehicle', columns='Year', values='VehicleCount')

In [66]:
# dfLP.loc[dfLP['Year'].isin([2000, 2010, 2018, 2030, 2050])]\
#     .loc[:, ['Year', 'Vehicle','VehicleCount']]\
#     .pivot(index='Vehicle', columns='Year', values='VehicleCount')